In [ ]:
# ================================
#  Setup: Clone the repository
# ================================
# 1. Move to /content (default Colab workspace)
# 2. Remove any old jewelry-enhancement folder (safe cleanup before rerun)
# 3. Clone the latest repo from GitHub
# 4. Print current working directory for confirmation

import os
os.chdir('/content')
# !rm -rf /content/jewelry-enhancement #use it when you are rerunning again
!git clone https://github.com/TejasCThakare/jewelry-enhancement.git
os.chdir('/content/jewelry-enhancement')
print(f" Dir: {os.getcwd()}")


In [ ]:
# ======================================
#  Install all Python dependencies
# ======================================
# Installs packages listed in requirements.txt

import os
os.chdir('/content/jewelry-enhancement')
!pip install -q -r requirements.txt
print(" Installed")


In [1]:
# ======================================
#  Setup Kaggle API key
# ======================================
# 1. Create Kaggle directory
# 2. Upload your kaggle.json file manually (from your local system)
# 3. It will prompt to ask that time upload that json

import os
os.chdir('/content/jewelry-enhancement')
!mkdir -p ~/.kaggle
from google.colab import files
uploaded = files.upload()
!cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
print(" Kaggle")


In [ ]:
# ======================================
#  Create required project directories
# ======================================
# Makes folders for raw data, degraded data, enhanced results,
# training splits, models, and logs/metrics.

import os
os.chdir('/content/jewelry-enhancement')
!mkdir -p data/{raw,degraded,enhanced,training/{train,val,test}} models results/{logs,metrics,comparisons}
print(" Dirs")


In [ ]:
# ======================================
# Download raw dataset from Kaggle
# ======================================
# Runs a script that uses Kaggle API to fetch jewelry images
# or any specified dataset defined in download_data.py.

import os
os.chdir('/content/jewelry-enhancement')
!python scripts/download_data.py


In [ ]:
# ======================================
#  Generate synthetic degraded images
# ======================================
# Applies blur, noise, and compression to create low-quality versions
# of the raw images for supervised enhancement training.

import os
os.chdir('/content/jewelry-enhancement')
!python scripts/create_degraded_dataset.py

In [ ]:
# ======================================
#  Run Real-ESRGAN enhancement on degraded images
# ======================================
# Uses pretrained Real-ESRGAN to produce enhanced outputs
# for comparison and further refinement.

import os
os.chdir('/content/jewelry-enhancement')
!python scripts/run_enhancement.py

In [ ]:
# ======================================
# Modify training data generation script for normalization
# ======================================
# Adds normalization step to convert images from [0,255] → [-1,1]
# before saving as NumPy arrays (for CNN stability).

import os
os.chdir('/content/jewelry-enhancement')
with open('scripts/generate_training_data.py') as f: c=f.read()
c=c.replace("np.save(output_path / f'{idx:04d}_input.npy', I_R)","np.save(output_path / f'{idx:04d}_input.npy', (I_R.astype(np.float32)/127.5)-1.0)")
c=c.replace("np.save(output_path / f'{idx:04d}_target.npy', I_HQ)","np.save(output_path / f'{idx:04d}_target.npy', (I_HQ.astype(np.float32)/127.5)-1.0)")
with open('scripts/generate_training_data.py','w') as f: f.write(c)
print(" Normalization added")


In [ ]:
# ======================================
#  Generate paired training data
# ======================================
# Creates (input, target) NumPy pairs for training the CNN refinement model.

import os
os.chdir('/content/jewelry-enhancement')
!rm -rf data/training/*
!python scripts/generate_training_data.py --max-images 490


In [ ]:
# ======================================
# Split dataset into train/val/test sets
# ======================================
# Organizes generated pairs into structured subfolders.

import os
os.chdir('/content/jewelry-enhancement')
!python src/training/dataset_split.py --source data/training


In [ ]:
# ======================================
# Sanity check: Verify normalization
# ======================================
# Loads one test sample and prints its min/max range.
# Expected range is close to [-1, 1].
# Prints "READY TO TRAIN" if normalization is correct.

import os, numpy as np
os.chdir('/content/jewelry-enhancement')
s=np.load('data/training/test/0000_input.npy')
print(f"Range: [{s.min():.2f}, {s.max():.2f}]")
print(" READY TO TRAIN" if -1.1<=s.min()<=s.max()<=1.1 else " ERROR")

from pathlib import Path
train_c = len(list(Path('data/training/train').glob('*_input.npy')))
val_c = len(list(Path('data/training/val').glob('*_input.npy')))
test_c = len(list(Path('data/training/test').glob('*_input.npy')))
print(f"Train: {train_c} | Val: {val_c} | Test: {test_c}")


In [ ]:
# ======================================
# Check dataset split counts
# ======================================
# Prints how many samples are present in each split
# (Train / Validation / Test).

from pathlib import Path
train_c = len(list(Path('data/training/train').glob('*_input.npy')))
val_c = len(list(Path('data/training/val').glob('*_input.npy')))
test_c = len(list(Path('data/training/test').glob('*_input.npy')))
print(f"Train: {train_c} | Val: {val_c} | Test: {test_c}")


In [ ]:
# ======================================
# Train the Refinement CNN model
# ======================================
# Trains the CNN-based refinement network for 20 epochs.
# Adjust --epochs and --batch-size as needed.
# --mixed-precision speeds up training on Colab GPUs.

import os
os.chdir('/content/jewelry-enhancement')
!python src/training/train_refinement_gan.py --mode cnn --epochs 20 --batch-size 4 --mixed-precision


In [ ]:
# ======================================
# Run demo / inference script
# ======================================
# After training, this launches a demo script that applies the
# trained model on sample jewelry images for enhancement.

!python /content/jewelry-enhancement/demo/app.py